In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.feature_selection import VarianceThreshold, SelectPercentile, SelectKBest, RFECV, f_regression 
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import  SVR


X_test = pd.read_csv("X_test.csv")
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")

columns = X_train.columns[1:]

# convert to np arrays
X = X_train.iloc[:,1:].to_numpy()
y = y_train.iloc[:,1:].to_numpy().reshape(-1)
X_test = X_test.iloc[:,1:].to_numpy()


x_stack = np.vstack((X,X_test))
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=15)
x_stack = imputer.fit_transform(x_stack)
X = x_stack[:1212]
X_test = x_stack[1212:]

outliers = IsolationForest(n_estimators=1000, random_state=0).fit_predict(X)
#behaviour='new', max_samples=100, random_state = 1, contamination='auto'
X = X[np.where(outliers == 1)[0]].copy()
y = y[np.where(outliers == 1)[0]].copy()

selector = VarianceThreshold()
selector.fit(X)
to_delete = np.where(selector.variances_ == 0)[0]
X = np.delete(X, to_delete, axis=1)
X_test = np.delete(X_test, to_delete, axis=1)

selector = SelectKBest(f_regression, k=94)
selector.fit(X, y)
X = selector.transform(X)
X_test = selector.transform(X_test)
indices = selector.get_support(indices=True)

import tensorflow as tf
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(X)

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from keras.layers.advanced_activations import LeakyReLU
from tensorflow import keras
from keras.layers import Dropout
from sklearn.metrics import r2_score
from keras import backend as K

def coeff_determination(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

model = Sequential(normalizer)
model.add(Dense(1024, input_dim=X.shape[1], activation=LeakyReLU(alpha=0.01)))
model.add(Dense(512, activation=LeakyReLU(alpha=0.01)))
model.add(Dropout(0.1,))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation=LeakyReLU(alpha=0.01)))
model.add(Dropout(0.1,))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.1,))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation=LeakyReLU(alpha=0.01)))
model.add(Dense(1, activation='relu'))

# Compile the network :
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mean_absolute_error', optimizer="adam", metrics=[coeff_determination])
print(model.summary())
model.fit(X, y, epochs=100, batch_size=32, validation_split = 0.2)

/home/tvinh/.local/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:294: RuntimeWarning: invalid value encountered in sqrt
  X_norms = np.sqrt(row_norms(X.T, squared=True) -


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 94)               189       
 n)                                                              
                                                                 
 dense (Dense)               (None, 1024)              97280     
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 1024)              525312    
                                                                 
 dense_3 (Dense)             (None, 1024)              1049600   
                                                        

In [2]:
print(X.shape)
print(X_test.shape)

(1209, 94)
(776, 94)


In [3]:
y_pred = model.predict(X_test)
X_test = pd.read_csv("X_test.csv")
df_ids = pd.DataFrame(X_test['id'])
df_predictions = df_ids.join(pd.DataFrame(y_pred, columns=['y']))
df_predictions.to_csv('model_results/nn_y_test.csv', index=False)

In [4]:
y_train_pred = model.predict(X)
X_train = pd.read_csv("X_train.csv")
df_ids = pd.DataFrame(X_train['id'])
df_predictions = df_ids.join(pd.DataFrame(y_train_pred, columns=['y']))
df_predictions.to_csv('model_results/nn_y_train.csv', index=False)

In [5]:
y_train_pred

array([[74.77643],
       [82.00549],
       [69.46909],
       ...,
       [78.33765],
       [71.97932],
       [76.14491]], dtype=float32)

In [6]:
y

array([71., 73., 66., ..., 82., 76., 81.])